# postcovid dataset preprocessing
### Import

## Train IMPACT model

In [69]:
from liriscat.utils import generate_eval_config
import json
from IMPACT.dataset import LoaderDataset as IMPACT_dataset
from liriscat.CDM import *
from IMPACT import model
from liriscat.dataset.preprocessing_utilities import *


folds_nb = 5
dataset_name="postcovid"

# Set all the required parameters ---------------
IMPACT_config = generate_eval_config(num_epochs=200, save_params=True, dataset_name=dataset_name, embs_path="../../embs/"+dataset_name, params_path="../../ckpt/"+dataset_name,  learning_rate=0.01885, lambda_=2e-7, batch_size=2048,valid_metric='mi_acc', pred_metrics=["mi_acc"])

concept_map = json.load(open(f'../2-preprocessed_data/{IMPACT_config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}
#parameter
metadata = json.load(open(f'../2-preprocessed_data/{IMPACT_config["dataset_name"]}_metadata.json', 'r'))
nb_modalities = torch.load(f'../2-preprocessed_data/{IMPACT_config["dataset_name"]}_nb_modalities.pkl',weights_only=True)

# Conversion helper that builds a DataFrame with specific dtypes and returns records.
def convert_to_records(data):
    df = pd.DataFrame(data, columns={'user_id': int, 'item_id': int, 'category_id': int,'correct': float, })
    return df.to_records(index=False, column_dtypes={'user_id': int, 'item_id': int, 'correct': float, 'category_id': int})

CUDA is not available. Using CPU.


In [70]:
dataset_name = 'postcovid'
for i in range(folds_nb):

    train = pd.read_csv(
    f'../2-preprocessed_data/{dataset_name}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
    valid= pd.read_csv(
    f'../2-preprocessed_data/{dataset_name}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
    # Merge the training and validation data for the current fold.
    train_valid_df = pd.concat([train, valid])
    # Split the merged data horizontally into train and validation sets.

    quadruplet = quadruplet_format(train_valid_df)
    train_data, valid_data, test_data = split_data_vertically(quadruplet, test_prop=0.2,valid_prop=0.1,folds_nb=5)

    # Convert each split in one step using the helper.
    horizontal_train = convert_to_records(train_data[0])
    horizontal_valid = convert_to_records(valid_data[0])
    horizontal_test = convert_to_records(test_data[0])

    impact_train_data = IMPACT_dataset(horizontal_train, concept_map, metadata, nb_modalities)
    impact_valid_data = IMPACT_dataset(horizontal_valid, concept_map, metadata, nb_modalities) # <---
    impact_test_data = IMPACT_dataset(horizontal_test, concept_map, metadata, nb_modalities)

    IMPACT_config['i_fold'] = i
    algo = model.IMPACT(**IMPACT_config)
    algo.init_model(impact_train_data, impact_valid_data)
    algo.train(impact_train_data, impact_valid_data)
    print(algo.evaluate_predictions(impact_test_data))

CUDA is not available. Skipping CUDA seed setting.
[INFO 24-50] train on cpu
[INFO 24-50] -- START Training --


 60%|██████    | 121/200 [01:03<00:41,  1.91it/s]

[INFO 25-53] -- END Training --
[INFO 25-53] Params saved


{'mi_acc': 0.8395308256149292, 'preds': tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), 'labels': tensor([1., 1., 1.,  ..., 2., 1., 1.], dtype=torch.float64)}
CUDA is not available. Skipping CUDA seed setting.
[INFO 25-57] train on cpu
[INFO 25-57] -- START Training --


 60%|██████    | 121/200 [00:58<00:38,  2.07it/s]

...

 60%|██████    | 121/200 [00:58<00:38,  2.07it/s]

[INFO 29-01] -- END Training --
[INFO 29-01] Params saved


{'mi_acc': 0.8395308256149292, 'preds': tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), 'labels': tensor([1., 1., 1.,  ..., 2., 1., 1.], dtype=torch.float64)}
CUDA is not available. Skipping CUDA seed setting.
[INFO 29-05] train on cpu
[INFO 29-05] -- START Training --


 60%|██████    | 121/200 [00:58<00:38,  2.07it/s]

[INFO 30-03] -- END Training --
[INFO 30-03] Params saved
{'mi_acc': 0.8395308256149292, 'preds': tensor([1., 1., 1.,  ..., 1., 1., 1.], dtype=torch.float64), 'labels': tensor([1., 1., 1.,  ..., 2., 1., 1.], dtype=torch.float64)}


In [61]:
quadruplet = quadruplet_format(train_valid_df)

valid_prop=0.1

df = pd.DataFrame(quadruplet, columns=["student_id", "item_id", "answer", "dimension_id"])
df.columns = ["student_id","item_id","answer","dimension_id"]
df.head()
df_grouped = df.groupby(['student_id', 'item_id']).agg(
    answers=('answer', list),
    dimensions=('dimension_id', list)
).reset_index()

#method of sci_ski_learn

for i_group, group in df_grouped.groupby('student_id'):
    group_idxs = np.array(group.index)
    train, valid = train_test_split(df_grouped, test_size=valid_prop, shuffle=True)
print("\nTraining set:")
print(train)

print("\nValidation set :")
print(valid)

train_expanded = train.explode(['answers', 'dimensions'])
valid_expanded = valid.explode(['answers', 'dimensions'])

print("\nTraining set after expanding :")
print(train_expanded)

print("\nValidation set after expanding :")
print(valid_expanded)


Training set:
       student_id  item_id                                            answers  \
11441         307      104                                              [1.0]   
27404         739       41  [1.6666666666666663, 1.6666666666666663, 1.666...   
2951           83        9                                              [1.0]   
19594         516       95                                              [2.0]   
11582         311       84                                              [1.5]   
...           ...      ...                                                ...   
18828         494       48  [1.6666666666666663, 1.6666666666666663, 1.666...   
6022          160        5                               [1.6666666666666663]   
17432         453       96                               [1.6666666666666663]   
26975         732       33                                    [2.0, 2.0, 2.0]   
11106         301        3                                              [2.0]   

      dimens

In [54]:
r_train = train_expanded.apply(lambda x : str(x['student_id']),axis=1).unique()
r_valid= valid_expanded.apply(lambda x : str(x['student_id']),axis=1).unique()

In [55]:
set(r_train) - (set(r_valid))

{'143',
 '189',
 '19',
 '234',
 '250',
 '29',
 '3',
 '321',
 '331',
 '361',
 '372',
 '4',
 '407',
 '422',
 '492',
 '535',
 '573',
 '574',
 '603',
 '61',
 '638',
 '663',
 '79'}

In [47]:
df_grouped.head()

,student_id,item_id,answers,dimensions
0,0,0,[1.3333333333333333],[0]
1,0,1,[2.0],[0]
2,0,2,[1.6666666666666663],[0]
3,0,3,[2.0],[0]
4,0,4,[1.3333333333333333],[0]


In [63]:
# Merge the training and validation data for the current fold.
train_valid_df = pd.concat([train[i], valid[i]])
# Split the merged data horizontally into train and validation sets.

quadruplet = quadruplet_format(train_valid_df)
train_data, valid_data, test_data = split_data_vertically(quadruplet, test_prop=0.2,valid_prop=0.1,folds_nb=5)

KeyError: 0

In [31]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import selectionStrategy
from liriscat import CDM

import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload
import IMPACT

import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(selectionStrategy)
reload(CDM)
reload(dataset)

config = utils.generate_eval_config(i_fold = 0,
                                    num_epochs=1,
                                    load_params=True,
                                    inner_user_lr=0.0001,
                                    esc = 'error',
                                    valid_metric= 'mi_acc',
                                    pred_metrics = ["mi_acc"],
                                    profile_metrics = ['doa'],
                                    save_params=False,
                                    n_query=5,
                                    batch_size=512)
utils.set_seed(config["seed"])

config["dataset_name"] = "postcovid"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../../embs/'+str(config["dataset_name"])
config['params_path']='../../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}outside model initialization
concept_map = json.load(open(f'../2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))

i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
train_data = dataset.CATDataset(train_df, concept_map, metadata, config, nb_modalities)
valid_data = dataset.EvalDataset(valid_df, concept_map, metadata, config, nb_modalities)
test_data = dataset.EvalDataset(test_df, concept_map, metadata, config, nb_modalities)

S = selectionStrategy.Random(metadata,**config)

CUDA is not available. Skipping CUDA seed setting.
CUDA is not available. Using CPU.
CUDA is not available. Skipping CUDA seed setting.
[INFO 36-38] postcovid
[INFO 36-38] #### postcovid ####
[INFO 36-38] #### config : {'seed': 0, 'dataset_name': 'postcovid', 'load_params': True, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.02026, 'batch_size': 512, 'valid_batch_size': 10000, 'num_epochs': 1, 'eval_freq': 1, 'patience': 30, 'device': device(type='cpu'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 5, 'CDM': 'impact', 'i_fold': 0, 'num_inner_users_epochs': 10, 'num_inner_epochs': 10, 'inner_lr': 0.0001, 'inner_user_lr': 0.0001, 'd_in': 4} ####
CUDA is not available. Skipping CUDA seed setting.
CUDA i

In [32]:
S.init_models(train_data, valid_data)

compiling CDM model
compiling selection model


In [33]:
S.evaluate_test(test_data)

/home/mbensafi/miniconda3/envs/IMPACT/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
  0%|          | 0/5 [00:01<?, ?it/s]


BackendCompilerFailed: backend='inductor' raised:
InvalidCxxCompiler: No working C++ compiler found in torch._inductor.config.cpp.cxx: (None, 'g++')

Set TORCH_LOGS="+dynamo" and TORCHDYNAMO_VERBOSE=1 for more information


You can suppress this exception and fall back to eager by setting:
    import torch._dynamo
    torch._dynamo.config.suppress_errors = True


In [5]:
torch.sum(S.CDM.model.R!=0)

tensor(0, device='cuda:0')